In [3]:
import chromadb
from langchain_classic.callbacks import AsyncIteratorCallbackHandler
from langchain_community.document_loaders import UnstructuredWordDocumentLoader
from langchain_ollama import ChatOllama
from langchain_ollama import OllamaEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma


llm = ChatOllama(
    model="llama3.1:8b",
)
embeddings = OllamaEmbeddings(model="embeddinggemma")

# vector_store = InMemoryVectorStore(embeddings)
vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    client=chromadb.HttpClient(host="http://localhost:9000"),
)
# vector_store = Chroma(
#     collection_name="example_collection",
#     embedding_function=embeddings,
#     persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
# )
vector_store._collection.count()

0

In [4]:
def add_document(path: str):
    loader = UnstructuredWordDocumentLoader(file_path=path)
    docs = loader.load()
    assert len(docs) == 1
    print(f"Total characters: {len(docs[0].page_content)}")

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,  # chunk size (characters)
        chunk_overlap=200,  # chunk overlap (characters)
        add_start_index=True,  # track index in original document
    )
    all_splits = text_splitter.split_documents(docs)
    print(f"Split blog post into {len(all_splits)} sub-documents.")

    document_ids = vector_store.add_documents(documents=all_splits)
    print(document_ids[:3])


In [7]:
add_document("data/Prepis_FG_PedagogLidr_mentori.docx")
# add_document("../../data/Prepis_FG_PedagogLidr_SKUPINA-B-android.docx")

Total characters: 45890
Split blog post into 64 sub-documents.
['2842bbeb-d8b7-4374-a289-b7986d5a19bc', '53f90d7b-3648-45f3-ba0f-74bc86c7a86d', 'e9c761ab-603d-47fc-9563-6f6862b57524']


In [3]:
from langchain.tools import tool

@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs


In [4]:
from langchain.agents import create_agent

tools = [retrieve_context]
# If desired, specify custom instructions
prompt = (
     "You are a helpful assistant designed to answer questions using only information retrieved "
    "from local document excerpts. You have access to a single tool called `retrieve_context`, "
    "which can fetch relevant text from local documents based on a query.\n\n"
    "Guidelines:\n"
    "1. Always start by using `retrieve_context` with the user's full question.\n"
    "2. If the returned context does not appear relevant, or does not fully answer the question, "
    "call `retrieve_context` again with a refined or different query that could produce better results.\n"
    "3. You have to call the tool multiple times in a single reasoning process — it’s encouraged.\n"
    "4. If, after multiple attempts, you still cannot find sufficient context, clearly state that the "
    "information is not available in the local documents.\n\n"
    "Answer in Czech unless the user explicitly requests another language."
)
agent = create_agent(llm, tools, system_prompt=prompt)

In [5]:
query = (
    "Co uvedla mluvčí číslo 2 o klasifikačním řádu?"
    # "Vnímají respondenti nějakou zodpovědnost šířit ve své škole to, co se naučily v programu? Co na to říká učitel číslo 3"
    # "Jak často do školy docházela mentorka?"
    # "Kolik dětí má mluvčí 1?",
    # "Co by pedagogičtí lídři uvítaly na další rok programu?"
)

for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Co uvedla mluvčí číslo 2 o klasifikačním řádu?
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (7d5a7df1-3446-4961-8ddc-6911294ec739)
 Call ID: 7d5a7df1-3446-4961-8ddc-6911294ec739
  Args:
    query: Co uvedla mluvčí číslo 2 o klasifikačním řádu?
================================= Tool Message =================================
Name: retrieve_context

Source: {'start_index': 34386, 'source': '../../data/Prepis_FG_PedagogLidr_mentori.docx'}
Content: 2: Také bývá, že má mít dvě známky z ústního zkoušení, bývá to součástí ŠVP, kde je klasifikační řád. 

3: Překážka může být i na straně rodičů. Najednou se tam děje něco, co je jiné, než znají. Nemám zprávy, že by šly nějaké informace směrem k rodičům, třeba o tom, jak zkouší jinou výuku v některých předmětech, to se přiznám, že nemají. Přijde mi, že to hodně zůstává uzavřené v třídách, možná v něja

In [14]:
for token, metadata in agent.stream(
        {"messages": [{"role": "user", "content": query}]},
        stream_mode="messages",
):
    node = metadata['langgraph_node']
    if len(token.content_blocks) > 0:
        content = token.content_blocks[0]
        type = content["type"] if "type" in content else None
        if type is None:
            continue

        if type == "text":
            # print(node)
            print(content["text"], end="")

Source: {'source': '../../data/Prepis_FG_PedagogLidr_mentori.docx', 'start_index': 34386}
Content: 2: Také bývá, že má mít dvě známky z ústního zkoušení, bývá to součástí ŠVP, kde je klasifikační řád. 

3: Překážka může být i na straně rodičů. Najednou se tam děje něco, co je jiné, než znají. Nemám zprávy, že by šly nějaké informace směrem k rodičům, třeba o tom, jak zkouší jinou výuku v některých předmětech, to se přiznám, že nemají. Přijde mi, že to hodně zůstává uzavřené v třídách, možná v nějakých menších kabinetech, že jsou takový opatrný.

Source: {'source': '../../data/Prepis_FG_PedagogLidr_mentori.docx', 'start_index': 40851}
Content: 2: Já vlastně zažívám, když přijdu do té hodiny u mentee, tak říkají, že jsem tam kvůli nim, protože se něco učí. Děti zažívají, že učitel se něco učí. Minimálně to slyší. 

(více lidí přitakává)

3: A my si pak o tom povídáme a hledáme, jak to dělat lépe. 

4: A to je podle mě doporučení, které mám z jiného projektu, ve kterém se pohybuji, tak ta